In [26]:
import os
import librosa
import numpy as np
import pandas as pd
import csv

# Path to the parent directory containing labeled folders
parent_dir = "../data/"

# Mapping labels to integers
label_mapping = {
    "angry": 0,
    "disgust": 1,
    "fear": 2,
    "happy": 3,
    "neutral": 4,
    "sad": 5,
    "surprise": 6
}

# Function to extract features from an audio file
def extract_features(audio_path, n_mfcc=13):
    y, sr = librosa.load(audio_path, sr=16000)  # Load and resample audio to 16 kHz
    features = {}
    
    # Extract MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc, axis=1)
    for i, val in enumerate(mfcc_mean):
        features[f"MFCC_{i+1}"] = val
    
    # Extract delta and delta-delta (first and second derivatives of MFCCs)
    delta_mfcc = librosa.feature.delta(mfcc)
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)
    for i, val in enumerate(np.mean(delta_mfcc, axis=1)):
        features[f"Delta_MFCC_{i+1}"] = val
    for i, val in enumerate(np.mean(delta2_mfcc, axis=1)):
        features[f"Delta2_MFCC_{i+1}"] = val
    
    # Extract prosodic features
    features["Energy"] = np.mean(librosa.feature.rms(y=y))  # Root Mean Square Energy
    features["Pitch_Mean"] = np.mean(librosa.pyin(y, fmin=librosa.note_to_hz('C2'),
                                                  fmax=librosa.note_to_hz('C7'), sr=sr)[0])  # F0 Mean
    
    # Extract spectral features
    features["Spectral_Centroid"] = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    features["Spectral_Bandwidth"] = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    features["Spectral_Rolloff"] = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
    features["Zero_Crossing_Rate"] = np.mean(librosa.feature.zero_crossing_rate(y=y))
    
    return features

# Output CSV file
output_csv = "optimized_extracted_features.csv"

# Open CSV file for writing
with open(output_csv, mode='w', newline='', encoding='utf-8') as csvfile:
    # Initialize CSV writer
    fieldnames = [f"MFCC_{i+1}" for i in range(13)] + \
                 [f"Delta_MFCC_{i+1}" for i in range(13)] + \
                 [f"Delta2_MFCC_{i+1}" for i in range(13)] + \
                 ["Energy", "Spectral_Centroid", 
                  "Spectral_Bandwidth", "Spectral_Rolloff", "Zero_Crossing_Rate", "Label"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()  # Write header

    # Loop through each folder and extract features
    for label in os.listdir(parent_dir):
        label_folder = os.path.join(parent_dir, label)
        if os.path.isdir(label_folder):  # Ensure it's a folder
            for file_name in os.listdir(label_folder):
                if file_name.endswith(".wav"):
                    file_path = os.path.join(label_folder, file_name)
                    # Extract features
                    features = extract_features(file_path)
                    features["Label"] = label_mapping[label.lower()]  # Add label
                    # Write features to CSV
                    writer.writerow(features)

print(f"Features extracted and directly saved to {output_csv}")


Features extracted and directly saved to optimized_extracted_features.csv
